In [1]:
import streamlit as st 
import json
import pandas as pd 
from Brawlstats import APIrequests
import base64
from datetime import  datetime
import mongomock
from pymongo import MongoClient

In [2]:
API_KEY = st.secrets['api']['api_key'] 
api =  APIrequests(api_key=API_KEY)

user_tag = 'RGG9ULLVU'
player_tag = user_tag
MAX_PLAYER_COUNT = {
    "soloShowdown": 10,
    "duoShowdown": 10,
    "roboRumble": 3,
    "bossFight": 3,
    "superCityRampage": 3,
    "gemGrab": 6,
    "brawlBall": 6,
    "bounty": 6,
    "heist": 6,
    "hotZone": 6,
    "knockout": 6,
}

In [3]:
def get_data(player_tag):
    player = api.get_player(player_tag)
    player = json.loads(player)
    player['battle_logs'] = json.loads(api.get_player_battle_log(player_tag))
    player['createdDatetime'] = datetime.now()
    return player

def recursive_items(data, results, parent_key=''):
    if isinstance(data, dict):
        for key, value in data.items():
            full_key = f"{parent_key}{key}" if parent_key == '' else f"{parent_key}_{key}"  # Create a full key
            yield from recursive_items(value, results, full_key)  # Recursively yield from the value
    elif isinstance(data, list):
        for idx, item in enumerate(data):
            # Create a new key for list items
            list_key = f"{parent_key}_{idx}" if parent_key else str(idx)
            yield from recursive_items(item, results, list_key)  # Recursively yield from each item in the list
    else:
        # Only yield if the value is not a dictionary
        if not isinstance(data, dict):
            yield parent_key, data  # Yield the parent key and value if it's neither a list nor a dict

def determine_win(row):
    return 1 if (row['result'] == 'victory' or row['rank'] <= row['max_player_count'] * 0.3) else 0

def get_played_brawlers(df, player_name):  
    played_brawlers = []
    for index, row in df.iterrows():
        if player_name in row.values:
            found_col = row.index[row.values == player_name ].tolist()
            lookup_column = found_col[0].replace('name','brawler_name')
            played_brawlers.append(row[lookup_column])
    return pd.DataFrame(played_brawlers, columns=['played_brawler'])

def calc_avg_game_duration(df):
    return df['duration'].mean()

def calc_win_ratio(df):
    if len(df) >= 2:
        wins_and_losses_values_count = df['isWin'].value_counts().to_list()
        return wins_and_losses_values_count[0]/(sum(wins_and_losses_values_count))    
    else:
        get_value = df['isWin'].values
        return float(get_value[0])

def calc_star_player_ratio(df, player_name):
    player_is_star_player = df.query(f"starPlayer_name == '{player_name}'")['starPlayer_name'].count()
    num_star_player_awards = df['starPlayer_name'].value_counts().sum()
    return player_is_star_player/num_star_player_awards

def calc_proportions(df):
    proportions = {}
    value_counts = df['played_brawler'].value_counts()
    brawler_names = value_counts.index
    sum_value_counts =sum(value_counts)
    brawler_picks = tuple(zip(brawler_names,value_counts))
    for brawler_name, pick_count in brawler_picks:
        pick_proportion = pick_count/sum_value_counts
        proportions[brawler_name] = [pick_count, pick_proportion]
    return proportions

def brawler_pick_rate(df):
    picks = calc_proportions(df)
    for brawler, pick_rate in picks.items():
        print(brawler, pick_rate[1])

In [4]:
player_tag ='RGG9ULLVU'
player_data = get_data(player_tag)#
player_name = player_data['name']

<Response [200]>
Data retrieved


# Construct Dataframes

In [5]:
df_brawlers = pd.DataFrame(player_data['brawlers'])
#df_brawlers

In [6]:
df_playerstats = pd.DataFrame(pd.json_normalize(player_data))
player_data_cols = df_playerstats.columns
df_playerstats = df_playerstats[player_data_cols
].drop(columns=['battle_logs.items','brawlers'])
#df_playerstats

In [17]:
battles = player_data['battle_logs']['items']

df_battles = pd.DataFrame(battles)
data = df_battles['event'] 
df_events = pd.DataFrame(data, columns=['event'])
flattened_df = pd.json_normalize(df_events['event'], sep='_')
#flattened_df
df_battle_brawler = df_battles['battle']
df_battle_brawler_flat = pd.json_normalize(df_battles['battle'], sep='_')
df_brawls = pd.concat([df_battles['battleTime'], flattened_df, df_battle_brawler_flat.drop(columns=['mode', 'teams'])],axis=1)
#df_brawls
all_df = pd.DataFrame(columns=df_brawls.columns)
all_battles = {}
for idx, battle in enumerate(df_battle_brawler.items()):
    results = {}
    for key, value in recursive_items(battle[1], results):
        results[key] = [value]
    battle_time = df_battles['battleTime'][idx]
    results['battleTime']=[battle_time]
    all_battles[battle_time] = results
    new_df = pd.DataFrame(all_battles[battle_time],index=[idx])
    uncommon_columns = list(set(df_brawls.columns).difference(set(new_df.columns)))
    sf_sll = pd.concat([new_df, df_brawls[uncommon_columns].iloc[[idx]]], axis=1)
    all_df=pd.concat([all_df, sf_sll], axis=0)
    
all_df = all_df.drop(columns='id')
all_df['duration'] = all_df['duration'].fillna(90)
if 'played_brawler' not in all_df.columns:
    #print('adding played_brawler column')
    played_brawlers = get_played_brawlers(all_df, player_name)
    all_df = pd.concat([all_df, played_brawlers], axis=1)
    
all_df = all_df.assign(max_player_count = all_df['mode'].map(MAX_PLAYER_COUNT))
all_df = all_df.assign(isWin = all_df.apply(determine_win, axis=1))
df_battles = all_df

# Calculate Aggregations and Statistics

In [18]:
df_battle_modes = df_battles[['mode', 'result','duration']]
df_battle_modes = pd.DataFrame(df_battle_modes.value_counts().reset_index())
df_battle_modes = df_battle_modes.rename(columns={0:'count'})
#df_battle_modes

In [19]:
brawlers_rank20 = df_brawlers.query('rank >=20')
brawlers_rank20 = brawlers_rank20[['name', 'rank']]
##print(brawlers_rank20)

win_rate_friendly = calc_win_ratio(all_df[all_df['type']=='friendly'])
##print(f"{win_rate_friendly = }")
win_rate_ranked = calc_win_ratio(all_df[all_df['type']=='ranked'])
#print(f"{win_rate_ranked = }")
win_rate_global = calc_win_ratio(all_df[['isWin']]) 
#print(f"{win_rate_global = }")

for brawler in played_brawlers.value_counts().index:
    brawler = brawler[0]
    df_brw = all_df[all_df['played_brawler']== brawler]
    brawler_wr = calc_win_ratio(df_brw)
    variable_name = f"win_rate_{brawler.lower()}"
    globals()[variable_name] = brawler_wr
    #print(variable_name, "=", globals()[variable_name])

In [39]:
def calc_rate_change(df):
    time_taken = df['duration'].sum() / 60 # minutes
    total_ressources = df['trophyChange'].sum()
    num_games = len(df['trophyChange'])
    print(f"{time_taken = :.0f} minutes, {total_ressources = }, {num_games = }")
    return (time_taken, total_ressources/num_games, total_ressources/time_taken)                

In [40]:
calc_rate_change(df)

time_taken = 36 minutes, total_ressources = 47, num_games = 25


(35.766666666666666, 1.88, 1.3140726933830382)